In [45]:
import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')
building = pd.read_csv('data/building_info.csv')

In [2]:
train.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


In [3]:
test.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77


In [4]:
building.head()

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-
1,2,건물기타,122233.47,99000.00,-,-,-
2,3,건물기타,171243.00,113950.00,40,-,-
3,4,건물기타,74312.98,34419.62,60,-,-
4,5,건물기타,205884.00,150000.00,-,2557,1000


## train eda

In [5]:
train[train['건물번호'] == 1]

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...,...
2035,1_20220824 19,1,20220824 19,25.5,NaN,2.9,63.0,0.1,0.19,2517.12
2036,1_20220824 20,1,20220824 20,25.0,NaN,2.6,66.0,0.0,0.01,2208.96
2037,1_20220824 21,1,20220824 21,24.6,NaN,1.3,67.0,NaN,NaN,1806.24
2038,1_20220824 22,1,20220824 22,24.1,NaN,2.1,70.0,NaN,NaN,1387.20


train전처리

In [6]:
train['hour'] = train['일시'].apply(lambda x : int(x[-2:]))
train['month'] = train['일시'].apply(lambda x : int(x[-7:-5]))
train['day'] = train['일시'].apply(lambda x : int(x[-5:-3]))
train = train.fillna(0)
train.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),hour,month,day
0,1_20220601 00,1,20220601 00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,0,6,1
1,1_20220601 01,1,20220601 01,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,1,6,1
2,1_20220601 02,1,20220601 02,17.7,0.0,1.5,45.0,0.0,0.0,974.88,2,6,1
3,1_20220601 03,1,20220601 03,16.7,0.0,1.4,48.0,0.0,0.0,953.76,3,6,1
4,1_20220601 04,1,20220601 04,18.4,0.0,2.8,43.0,0.0,0.0,986.40,4,6,1


In [7]:
from lightgbm import LGBMRegressor

In [18]:
from sklearn.model_selection import train_test_split

train_x = train[['건물번호', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)','hour','month','day']]
train_y = train['전력소비량(kWh)']

train_X, valid_X, train_Y, valid_Y = train_test_split(train_x, train_y, test_size = 0.2, random_state = 42)

In [24]:
model = LGBMRegressor(n_estimators=1000)

In [25]:
eval_set = [(valid_X,valid_Y)]
model.fit(train_X,train_Y,  eval_metric = 'mae',
          eval_set = eval_set)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006699 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 791
[LightGBM] [Info] Number of data points in the train set: 163200, number of used features: 8
[LightGBM] [Info] Start training from score 2453.452806


LGBMRegressor(n_estimators=1000)

In [46]:
test['hour'] = test['일시'].apply(lambda x : int(x[-2:]))
test['month'] = test['일시'].apply(lambda x : int(x[-7:-5]))
test['day'] = test['일시'].apply(lambda x : int(x[-5:-3]))
test = test.fillna(0)
test.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),hour,month,day
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72,0,8,25
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72,1,8,25
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75,2,8,25
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78,3,8,25
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77,4,8,25


In [47]:
test_x = test[['건물번호', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)','hour','month','day']]

In [48]:
preds = model.predict(test_x)

In [49]:
submission['answer'] = preds

In [50]:
submission.to_csv('first_answer.csv',index = False)